**Indexes Definitions**

 - Let $I$ represent the Class
 - Let $J$ represent the Classrooms
 - Let $K$ represent all the time slots available from 8:00am to 9:30pm for each week
     - *Here we assume each time slot is 1.5 hours, and the duration of each class is 1.5 hours as well*
 

**Variable Definitions**
 
 - $X_{ijk}$ is a binary indicating whether course $i$ is assigned to classroom $j$ in time slot $k$
 - $C_i$ is the number of seats (or max. number of students) in class $i$
 - $R_j$ is the capacity of classroom $j$
 - $Max.U$ is the maximum of the sum of $C_iX_{ijk}$ for every $j \in j$ and every $k \in k$. It's the maximum utilization rate.
 - $Min.U$ is the minimum of the sum of $C_iX_{ijk}$ for every $j \in j$ and every $k \in k$. It's the minimum utilization rate.

 
**Objective Function**
 
 - The Objective is to minimize the function:  
 ${Max.U-Min.U}$
 
**Constraints**

Each class gets 1 slot in 1 classroom:  
$\sum_{j \in J,k \in K}X_{ijk}=1\quad\text{for every $i \in I$}$

Each classroom should have no more than 1 class in each timeslot:  
$\sum_{i \in I}X_{ijk}\le 1\quad\text{for every $j \in J$ and every $k \in K$}$

Number of seats in a class cannot exceed classroom capacity:  
$\sum_{i \in I}C_iX_{ijk}\le R_j\quad\text{for every $j \in J$ and every $k \in K$}$



In [1]:
import gurobipy as grb
import pandas as pd
data=pd.read_excel('input_data.xlsx',sheetname=None,index_col=0)
rooms=data['rooms']
sections=data['sections']

In [2]:
rooms.head()

,Size
Room,
ACC 306B,120
ACC201,200
ACC205,50


In [3]:
sections.head()

,Seats
Section,
14029,170
14027,40
14026,41
14028,44
14025,45


In [2]:
I = sections.index #List of section id's
J = rooms.index #List of room numbers
K = range(10) # 84min blocks 0...49 from 8am Monday to 10pm Friday
C = sections.Seats
R = rooms.Size

mod=grb.Model()
#Define decision variables (section, room, time block)
X={}
for i in I:
    for j in J:
        for k in K:
            X[i,j,k]=mod.addVar(vtype=grb.GRB.BINARY, name = 'x[{0},{1},{2}]'.format(i,j,k))

In [4]:
R

Room
ACC 306B    120
ACC201      200
ACC205       50
Name: Size, dtype: int64

In [5]:
###Constraints###

#Each section must be assigned one class block
for i in I:
    mod.addConstr(sum(X[i,j,k] for j in J for k in K) == 1)
    
#Each classroom, time block can't be assigned more than 1 section
for j in J:
    for k in K:
        mod.addConstr(sum(X[i,j,k] for i in I) <= 1)
        
#Rooms must have enough capacity for seats in section
for j in J:
    for k in K:
        mod.addConstr(sum(C[i]*X[i,j,k] for i in I) - R[j] <= 0)

#Upper and Lower Bound vars for Objective function (classroom-timeblock utlizations)
U = mod.addVar(name = 'Upper Util')
L = mod.addVar(name = 'Lower Util')

# Add U and L constraint for each classroom and time block
for j in J:
    for k in K:
        mod.addConstr(sum(C[i]/(R[j]*1.0)*X[i,j,k] for i in I) <= U)
        mod.addConstr(sum(C[i]/(R[j]*1.0)*X[i,j,k] for i in I) >= L)

In [6]:
#Objective Function - Minimize distance between U and L#
mod.setObjective(U-L, sense=grb.GRB.MINIMIZE)

In [7]:
mod.setParam('OutputFlag',False)   
mod.optimize()
print('Optimal objective: {0:.2f}'.format(mod.ObjVal))

Optimal objective: 0.62


In [8]:
L.x

0.23

In [9]:
U.x

0.85

In [64]:
mod.getVars()

[<gurobi.Var x[14029,ACC 306B,0] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,1] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,2] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,3] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,4] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,5] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,6] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,7] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,8] (value 0.0)>,
 <gurobi.Var x[14029,ACC 306B,9] (value 0.0)>,
 <gurobi.Var x[14029,ACC201,0] (value 0.0)>,
 <gurobi.Var x[14029,ACC201,1] (value 0.0)>,
 <gurobi.Var x[14029,ACC201,2] (value -0.0)>,
 <gurobi.Var x[14029,ACC201,3] (value -0.0)>,
 <gurobi.Var x[14029,ACC201,4] (value 0.0)>,
 <gurobi.Var x[14029,ACC201,5] (value -0.0)>,
 <gurobi.Var x[14029,ACC201,6] (value 1.0)>,
 <gurobi.Var x[14029,ACC201,7] (value -0.0)>,
 <gurobi.Var x[14029,ACC201,8] (value -0.0)>,
 <gurobi.Var x[14029,ACC201,9] (value -0.0)>,
 <gurobi.Var x[14029,ACC205,0] (value 0.0)>,
 <gurobi.Var x[14029,ACC205,1

In [43]:
sections.loc[14044,'Seats']

170